The purpose of this notebook is to test the speed of diffent ABSA approaches

In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


c:\Users\jacop\Desktop\Lavori\Consigl_IA_mi-\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Testing the model on a single review

In [ ]:
bos_instruction = """Definition: The output will be the aspects (both implicit and explicit) and the aspects sentiment polarity. In cases where there are no aspects the output should be noaspectterm:none.
    Positive example 1-
    input: I charge it at night and skip taking the cord with me because of the good battery life.
    output: battery life:positive, 
    Positive example 2-
    input: I even got my teenage son one, because of the features that it offers, like, iChat, Photobooth, garage band and more!.
    output: features:positive, iChat:positive, Photobooth:positive, garage band:positive
    Negative example 1-
    input: Speaking of the browser, it too has problems.
    output: browser:negative
    Negative example 2-
    input: The keyboard is too slick.
    output: keyboard:negative
    Neutral example 1-
    input: I took it back for an Asus and same thing- blue screen which required me to remove the battery to reset.
    output: battery:neutral
    Neutral example 2-
    input: Nightly my computer defrags itself and runs a virus scan.
    output: virus scan:neutral
    Now complete the following example-
    input: """
delim_instruct = ''
eos_instruct = ' \noutput:'
text = 'The food here is sooooo good!! So good and tasty. Yaaaam! We had the best sangria here. We tried sangria in many places but here was the best one.  Big portions of food, good prices, very tasty. We ate here more than one time so I definitely recommend this place'

tokenizer = AutoTokenizer.from_pretrained("kevinscaria/joint_tk-instruct-base-def-pos-neg-neut-combined")
model = AutoModelForSeq2SeqLM.from_pretrained("kevinscaria/joint_tk-instruct-base-def-pos-neg-neut-combined")

tokenized_text = tokenizer(bos_instruction + text + delim_instruct + eos_instruct, return_tensors="pt")
output = model.generate(tokenized_text.input_ids)
print('Model output: ', tokenizer.decode(output[0], skip_special_tokens=True))

Model output:  food:positive, sangria:positive, prices:positive


In [5]:
tokenizer.decode(output[0], skip_special_tokens=True)

'food:positive, sangria:positive, prices:positive'

### Testing the model on 100 observations


In [2]:
import pandas as pd 

path = "c:\\Users\\jacop\\Desktop\\Lavori\\Consigl_IA_mi-\\data\\raw\\Barcelona_reviews.csv"
raw_data = pd.read_csv(path)



test_data = raw_data.head(100)


C:\Users\jacop\AppData\Local\Temp\ipykernel_48824\1248827139.py:4: DtypeWarning: Columns (0,1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv(path)


In [18]:
def absa_test(df, model, tokenizer):
    # New dataset
    absa_rows = []
    malformed_item = 0 #number of aspect with errors

    for idx, row in df.iterrows():
        original_review = row['review_full']
        #print(f'ORIGINAL REVIEW : {original_review}')
        
        if idx%10 == 0:
            print(f'Progress: {idx}/{len(df)}')
        
        
        
        bos_instruction = """Definition: The output will be the aspects (both implicit and explicit) and the aspects sentiment polarity. In cases where there are no aspects the output should be noaspectterm:none.
        Positive example 1-
        input: I charge it at night and skip taking the cord with me because of the good battery life.
        output: battery life:positive, 
        Positive example 2-
        input: I even got my teenage son one, because of the features that it offers, like, iChat, Photobooth, garage band and more!.
        output: features:positive, iChat:positive, Photobooth:positive, garage band:positive
        Negative example 1-
        input: Speaking of the browser, it too has problems.
        output: browser:negative
        Negative example 2-
        input: The keyboard is too slick.
        output: keyboard:negative
        Neutral example 1-
        input: I took it back for an Asus and same thing- blue screen which required me to remove the battery to reset.
        output: battery:neutral
        Neutral example 2-
        input: Nightly my computer defrags itself and runs a virus scan.
        output: virus scan:neutral
        Now complete the following example-
        input: """
        delim_instruct = ''
        eos_instruct = ' \noutput:'
        text = original_review
        
        tokenized_text = tokenizer(bos_instruction + text + delim_instruct + eos_instruct, return_tensors="pt")
        output = model.generate(tokenized_text.input_ids)
        print('Model output: ', tokenizer.decode(output[0], skip_special_tokens=True))
        s = tokenizer.decode(output[0], skip_special_tokens=True)
        
        d = {}
        for item in s.split(","):
            parts = item.strip().split(":")
            if len(parts) == 2:
                key, value = parts
                d[key.strip()] = value.strip()
            else:
                print(f"⚠️ Skipped malformed item: {item.strip()} in review number {idx}")
                malformed_item += 1
        
        
        absa_rows.append({
            "restaurant_name": row['restaurant_name'],
            "review_id": row['review_id'],
            "review": original_review, 
            "rating_review": row['rating_review'],
            "city": row['city'],
            "date": row['date'],
            "aspects": d 
            
        })
    print(f'Malformed outpuds: {malformed_item}')
    return absa_rows
        #print('##################  Added new review to absa_df  ############################')



In [19]:
tokenizer = AutoTokenizer.from_pretrained("kevinscaria/joint_tk-instruct-base-def-pos-neg-neut-combined")
model = AutoModelForSeq2SeqLM.from_pretrained("kevinscaria/joint_tk-instruct-base-def-pos-neg-neut-combined")

absa_results = absa_test(test_data, model, tokenizer )

Progress: 0/100
Model output:  service:negative, food:negative, service:negative, food:negative
Model output:  food:positive, desserts:positive, price:positive, tapas:positive
Model output:  food:positive, ambiance:positive, price:positive, crew:positive
Model output:  food:positive, price:positive, crew:positive, service:positive, abiance:
Model output:  food:positive, service:positive
Model output:  environment:positive, service:positive, menu choices:positive, food:positive, prices:positive
Model output:  environment:positive, service:positive, menu choices:positive, food:positive, prices:positive


Token indices sequence length is longer than the specified maximum sequence length for this model (594 > 512). Running this sequence through the model will result in indexing errors


Model output:  food:positive, atmosphere:positive, waitress:positive
Model output:  food:positive, staff:negative, customer service:negative
Model output:  food:positive, sangria:positive, prices:positive
Progress: 10/100
Model output:  seafood pasta:positive, ingredients:positive
Model output:  food:positive, service:positive
Model output:  italian pasta:positive, foods:positive
Model output:  tiramisu:positive, coffee flavor:negative, mascarpone richness:
Model output:  service:positive, quality of the pizza:positive, Marechiaro:positive, product:
Model output:  Italian food:positive, pizzas:positive, foccacia:positive, vegetable and meat la
⚠️ Skipped malformed item: vegetable and meat la in review number 15
Model output:  place:positive, decor:positive, Waiters:positive, meal:neutral, spanish
⚠️ Skipped malformed item: spanish in review number 16
Model output:  italian wine:positive, Blange:positive, seafood:positive, sauce:positive, bread:
Model output:  napolitanian pizza:negativ

In [20]:
from collections import Counter
from tabulate import tabulate  # Make sure you have this installed

#Counts hown many different aspects do we have
def count_aspect(absa_results):
    # Estrai tutte le chiavi dai dizionari contenuti nella chiave 'aspect'
    all_aspect = []

    for item in absa_results:
        aspects = item.get('aspects', {})
        all_aspect.extend(aspects.keys())
        unique_aspects = set(all_aspect)
        num_unique_aspects = len(all_aspect)

    # Conta le occorrenze di ciascuna chiave
    aspect_counts = Counter(all_aspect)

    # Risultato
    table = tabulate(aspect_counts.items(), headers=["Aspect", "Count"], tablefmt="grid")
    print(f'Number of different aspects identified: {num_unique_aspects}')
    print(table)



In [21]:
count_aspect(absa_results)

Number of different aspects identified: 321
+-------------------------------------------+---------+
| Aspect                                    |   Count |
+===========================================+=========+
| service                                   |      47 |
+-------------------------------------------+---------+
| food                                      |      58 |
+-------------------------------------------+---------+
| desserts                                  |       1 |
+-------------------------------------------+---------+
| price                                     |       7 |
+-------------------------------------------+---------+
| tapas                                     |       7 |
+-------------------------------------------+---------+
| ambiance                                  |       2 |
+-------------------------------------------+---------+
| crew                                      |       2 |
+-------------------------------------------+---------+
| ab

Using the pretrained model 'kevinscaria/joint_tk-instruct-base-def-pos-neg-neut-combined' from Hugging face it took 1.40 min for the first 100 reviews

## Using fine tuned model by Iceland



In [15]:
tokenizer_Ice = AutoTokenizer.from_pretrained("Iceland/pyabsa-v3-onlyRest")
model_Ice = AutoModelForSeq2SeqLM.from_pretrained("Iceland/pyabsa-v3-onlyRest")



In [22]:
absa_test_Ice = absa_test(test_data, model_Ice, tokenizer_Ice )

Progress: 0/100
Model output:  service:negative, food:negative
Model output:  food:positive, desserts:positive
Model output:  food:positive, ambiance:positive, price:positive, crew:positive
Model output:  food:positive
Model output:  food:positive, service:positive
Model output:  environment:positive, service:positive, menu choices:positive, food:positive, prices:positive
Model output:  environment:positive, service:positive, menu choices:positive, food:positive, prices:positive


Token indices sequence length is longer than the specified maximum sequence length for this model (594 > 512). Running this sequence through the model will result in indexing errors


Model output:  food:positive, atmosphere:positive
Model output:  food:positive
Model output:  food:positive
Progress: 10/100
Model output:  seafood pasta:positive
Model output:  sports bar:positive
Model output:  servings of italian pasta:positive, foods:positive
Model output:  tiramisu:negative
Model output:  service:positive, quality of the pizza:positive, staff:positive
Model output:  pizzas:positive, foccacia:positive, vegetable and meat lasagne:positive
Model output:  decor:positive, Waiters:positive, meal:positive
Model output:  food:positive, service:positive
Model output:  napolitanian pizza:negative
Model output:  service:positive, quality of the ingredients:positive, Red wines:positive
Progress: 20/100
Model output:  menu:negative, service:negative, waiter:negative
Model output:  server:positive, Kate:positive, wine list:positive
Model output:  food:positive
Model output:  food:positive, atmospher:positive, staff:positive
Model output:  service:positive
Model output:  food:po

In [23]:
count_aspect(absa_test_Ice)

Number of different aspects identified: 236
+-----------------------------------------+---------+
| Aspect                                  |   Count |
+=========================================+=========+
| service                                 |      42 |
+-----------------------------------------+---------+
| food                                    |      45 |
+-----------------------------------------+---------+
| desserts                                |       1 |
+-----------------------------------------+---------+
| ambiance                                |       2 |
+-----------------------------------------+---------+
| price                                   |       4 |
+-----------------------------------------+---------+
| crew                                    |       1 |
+-----------------------------------------+---------+
| environment                             |       2 |
+-----------------------------------------+---------+
| menu choices                        

# Using LLM

In [ ]:
import sys
sys.path.append('c:\\Users\\jacop\\Desktop\\Lavori\\Consigl_IA_mi-\\scripts')  

In [6]:
df  = absa(test_data)

Progress: 0/100
Progress: 10/100
Progress: 20/100
Progress: 30/100
Progress: 40/100
Progress: 50/100
Progress: 60/100
Progress: 70/100
Progress: 80/100
Progress: 90/100


OSError: Cannot save file into a non-existent directory: 'data\processed'

Using Mistral 7B in took 6.20 min for the first 100 reviews.